## 新疆棉事件

导入写好的程序，对数据进行清洗、分析和统计，得到关键元素和关键指标

In [1]:
import pandas as pd
from src.process.data_process import sentiment_analysis, Process
from src.process.plot_echarts import plot_bar, plot_base_pie, plot_multi_pie, plot_them_river, plot_wordcloud
from src.process.plot_echarts import plot_stack_bar, plot_stack_pie, plot_line, plot_heatmap

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NotebookType = NotebookType.JUPYTER_NOTEBOOK
df = pd.read_excel('./data/zhanshi.xlsx')
p = Process(df)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\STUDYH~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built succesfully.



使用自然语言处理中的深度学习算法distilbert-base-multilingual-cased-sentiments，对数据进行情感倾向的识别。

得到每条数据的情感倾向，并进行统计

情感倾向柱状图

In [2]:
pic = plot_bar(p.sentiment_count)
pic.render_notebook()

情感倾向饼图

In [3]:
pic = plot_base_pie(p.sentiment_count)
pic.render_notebook()

从图中可以看出，大多数新闻呈正向情绪，对新疆棉充满信心，负向情绪少于正向情绪，中立情绪较少。

爬取数据来自网络，同时我们对数据来源也做了统计，由于来源冗杂，我们为了抓取主要结论，仅对头部来源进行展示。

In [4]:
pic = plot_multi_pie(p.sentiment_count, p.source_dict)
pic.render_notebook()

可以看到数据分布相对平均，我们按照官媒和网媒进行划分，重新对数据进行统计

In [5]:
pic = plot_stack_bar(p.source_sentiment_count)
pic.render_notebook()

In [6]:
pic = plot_stack_pie(p.source_sentiment_pie)
pic.render_notebook()

可以看出无论是网媒还是官媒，分布比例都是一致的，大多数新闻呈正向情绪，负向情绪少于正向情绪，中立情绪较少。该结论较为可信

将时间加入考虑，分析情感变化的整体趋势

河流图

In [7]:
p.df['date'] = p.df['发布时间'].apply(lambda x: str(x)[:10])
data = df.groupby(['date', '情感倾向']).count()['text'].reset_index()
pic = plot_them_river(list(data['情感倾向'].unique()), data[['date', 'text', '情感倾向']].values.tolist())
pic.render_notebook()

折线图

In [8]:
pic = plot_line(p.date_count)
pic.render_notebook()

热力图

In [9]:
row, col, data = p.heat_count
pic = plot_heatmap(row, col, data)
pic.render_notebook()

对文本数据进行分词，去除stop words后，使用TF-IDF算法对词赋予权重，并根据权重进行LDA主题分析

In [10]:
p.lda_summary('情感中立', 10, 5)

情感中立 LDA主题分析关键词提取：
主题 1: 病例 确诊 新增 无症状 感染者
主题 2: 天鹅 人權 湿地 伊宁县 仅供
主题 3: 经销商 二手车 商家 爱卡 涉及
主题 4: 索尼 地震 委托 强制 现象
主题 5: 桥梁 照明 最低工资 夜景 灯光
主题 6: 补贴 美国 劳动者 欧盟 林肯
主题 7: 阿克苏市 滴灌 货运 价格 生猪
主题 8: db 几日 截图 长图 辛辛苦苦
主题 9: 新疆 大盘鸡 病案 巴宝莉 财经
主题 10: 中国 棉花 bci 新疆 华春莹


In [11]:
p.lda_summary('情感负向', 10, 5)

情感负向 LDA主题分析关键词提取：
主题 1: hm 共青团中央 新疆 棉花 碰瓷
主题 2: 地震 拜城县 房屋 65 受损
主题 3: 对不起 访问 新闻资讯 页面 智能
主题 4: 新疆 共青团 欧盟 棉被 有句
主题 5: 大赏 迷惑 崔天凯 一大堆 中國
主题 6: 米粉 地狱 新疆 原创 微博
主题 7: 新疆 美国 人权 中国 发展
主题 8: 棉花 新疆 中国 amp bci
主题 9: 新疆 棉花 中国 企业 万吨
主题 10: 最低工资 信息 招标 劳动者 customer


In [12]:
p.lda_summary('情感正向', 10, 5)

情感正向 LDA主题分析关键词提取：
主题 1: 地震 项目 招标 nbsp 公告
主题 2: 棉花 bci 新疆 中国 声明
主题 3: 棉花 新疆 支持 品牌 中国
主题 4: 劳动 工作 教育 公司 nbsp
主题 5: 新疆 学习 发展 党史 工作
主题 6: 新疆 共青团 棉花 微博 有句
主题 7: 毛巾 观察者 网送 hm 一套
主题 8: 棉花 万吨 产量 新疆 国内
主题 9: 青岛 男篮 比赛 新疆 蛋蛋
主题 10: 杏花 吐鲁番 新疆 乌鲁木齐 地震


In [13]:
p.lda_summary('总体')

总体 LDA主题分析关键词提取：
主题 1: 新疆 棉花 中国 华春莹 企业
主题 2: 劳动 教育 雪白 工作 学习
主题 3: 棉花 新疆 中国 amp bci
主题 4: 新疆 共青团 微博 hm 棉花
主题 5: 地震 新疆 棉花 招标 项目


词云图

In [14]:
src = p.words
pic = plot_wordcloud(list(zip(src.keys(), src.values())))
pic.render_notebook()